### Using the [PyPortfolioOpt library](https://github.com/robertmartin8/PyPortfolioOpt) library to generate an optimzied portfolio from the S&P 500 list of companies
Reference: [Build A Killer Stock Portfolio Using Python](https://www.youtube.com/watch?v=bvDkel5whUY&t=2s&ab_channel=ComputerScience)

<b><font color="red">Click <a href="https://colab.research.google.com/github/ebharucha/Portfolio-Optimization/blob/master/PortfolioOpt.ipynb" target="#">here</a> to open/run the notebook in Google Colab</font></b>

@ebharucha 12/31/2020

### Install & import dependencies

In [2]:
!pip install PyPortfolioOpt
!pip install pulp

In [103]:
import pandas as pd
import pandas_datareader as web
import datetime
from dateutil.relativedelta import relativedelta
import os
import shutil
import pickle
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import risk_models, expected_returns
import warnings
warnings.filterwarnings("ignore")

### Get & initialize data

#### <font color="Red">Specify value of overall portfolio in USD</font>

In [2]:
portfolio_val = 50000

#### Get S&P 500 companies

In [3]:
DATADIR = './data'
if not os.path.exists(DATADIR):
    os.makedirs(DATADIR)

table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df_SP500_ = table[0]
df_SP500_.Symbol = df_SP500_.Symbol.replace('\.','-', regex=True)
df_SP500_.to_csv(f'{DATADIR}/S&P500-Info.csv')
df_SP500_.to_csv(f'{DATADIR}/S&P500-Symbols.csv', columns=['Symbol'])

#### Load daily closing prices for S&P 500 components over past 10 years in a DataFrame

<i><font color="Red">This will take some time to run</font></i>

In [180]:
symbols = df_SP500_.Symbol

PKLFILE = 'SP500.pkl'
shutil.copyfile(f'{DATADIR}/{PKLFILE}', f'{DATADIR}/{PKLFILE}.bak')  # Make a backup copy

today = str(datetime.date.today())
f = lambda sym, start: web.get_data_yahoo(sym,
                            start = start,
                            end = today)['Adj Close']

# Load fromdata pickle file if it exists
if os.path.isfile(f'{DATADIR}/{PKLFILE}'):
    try:
        with open(f'{DATADIR}/{PKLFILE}', 'rb') as pklfile:
            df_SP500 = pickle.load(pklfile)
    except:
        print (f'Failed to load {DATADIR}/{PKLFILE}')
    last = df_SP500.iloc[-1:].index[0].to_pydatetime().date()
    if (str(last + relativedelta(days=+1)) != today):
        start = str(last + relativedelta(days=+1))
        print (f'Fetching missing data from {start} to {today}')
        df_tmp = pd.DataFrame(columns=symbols)
        df_tmp = f(symbols, start)
        df_tmp = df_tmp.iloc[1:,:]
        df_SP500 = pd.concat([df_SP500, df_tmp], axis=0)
else:
    current_year = datetime.datetime.now().year
    start = (f'{current_year-10}-01-01')
    df_SP500 = pd.DataFrame(columns=symbols)
    df_SP500 = f(symbols, start)
    
# Pickle the data
with open (f'{DATADIR}/SP500.pkl', 'wb') as pklfile:
    pickle.dump(df_SP500, pklfile)

#### <font color="red">Specify how many years of past data to use  default is 10</font>

In [181]:
# Specify how many yers of past data to use 
no_of_years = 5  # default value is 10, which is the furthest back you can go
current_year = datetime.datetime.now().year
start_date = f'{current_year-no_of_years-1}-12-31'

df_SP500 = df_SP500.loc[start_date:]

#### Get latest prices

In [193]:
latest_prices = get_latest_prices(df_SP500)

### Portfolio optimization

#### Expected annualized returns & annualized covariance matrix of the daily asset returns

In [194]:
mu = expected_returns.mean_historical_return(df_SP500)
S = risk_models.sample_cov(df_SP500)

#### Optimzie for the maximal Sharpe ratio

In [195]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
weights = cleaned_weights
# print (cleaned_weights)
ef.portfolio_performance(verbose=True)

Expected annual return: 136.7%
Annual volatility: 25.5%
Sharpe Ratio: 5.28


(1.3672991690223297, 0.2552026379747216, 5.279330886680658)

#### Get stock allocations

In [196]:
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=portfolio_val)
allocation, leftover = da.lp_portfolio()
print (f'Discrete share allocations: {allocation}')
print (f'Funds remaining: ${leftover:.2f}')

Discrete share allocations: {'AMD': 34, 'CARR': 365, 'DPZ': 12, 'ETSY': 3, 'NEM': 80, 'POOL': 11, 'TTWO': 9, 'FTI': 2, 'VNT': 351, 'WMT': 38}
Funds remaining: $0.10


In [197]:
# Get company name & sector
name = lambda sym: df_SP500_[df_SP500_.Symbol == sym].Security.values[0]
sector_ = lambda sym: df_SP500_[df_SP500_.Symbol == sym]['GICS Sector'].values[0]

name('NFLX'), sector_('NFLX')

('Netflix Inc.', 'Communication Services')

#### Construct portfolio DataFrame

In [187]:
port_df = pd.DataFrame(columns=['Symbol', 'Company Name', "Sector", "Price", "No. of Shares", "Allocation($)"])

In [188]:
symbols = allocation.keys()
company_name = []
sector = []
price = []
no_of_shares = []
allocation_dollar = []
total_allocation = 0

for sym in symbols:
    company_name.append(name(sym))
    sector.append(sector_(sym))
    price.append(f'{latest_prices[sym]:,.2f}')
    no_of_shares.append(allocation.get(sym))
    allocation_dollar.append(f'{allocation.get(sym) * latest_prices[sym]:,.2f}')
    total_allocation = total_allocation + allocation.get(sym) * latest_prices[sym]

In [189]:
port_df['Symbol'] = symbols
port_df['Company Name'] = company_name
port_df['Sector'] = sector
port_df['Price'] = price
port_df['No. of Shares'] = no_of_shares
port_df['Allocation($)'] = allocation_dollar

### <font color="red">Display portfolio allocations & expected performance</dont>

In [190]:
port_df

,Symbol,Company Name,Sector,Price,No. of Shares,Allocation($)
0,AMD,Advanced Micro Devices Inc,Information Technology,91.71,34,"3,118.14"
1,CARR,Carrier Global,Industrials,37.72,365,"13,767.80"
2,DPZ,Domino's Pizza,Consumer Discretionary,383.46,12,"4,601.52"
3,ETSY,Etsy,Consumer Discretionary,177.91,3,533.73
4,NEM,Newmont Corporation,Materials,59.89,80,"4,791.20"
5,POOL,Pool Corporation,Consumer Discretionary,372.50,11,"4,097.50"
6,TTWO,Take-Two Interactive,Communication Services,207.79,9,"1,870.11"
7,FTI,TechnipFMC,Energy,9.40,2,18.80
8,VNT,Vontier,Information Technology,33.40,351,"11,723.40"
9,WMT,Walmart,Consumer Staples,144.15,38,"5,477.70"


In [179]:
port_df

,Symbol,Company Name,Sector,Price,No. of Shares,Allocation($)
0,ATVI,Activision Blizzard,Communication Services,92.85,4,371.40
1,CARR,Carrier Global,Industrials,37.72,387,"14,597.64"
2,CHTR,Charter Communications,Communication Services,661.55,2,"1,323.10"
3,CLX,The Clorox Company,Consumer Staples,201.92,18,"3,634.56"
4,DG,Dollar General,Consumer Discretionary,210.30,5,"1,051.50"
5,DPZ,Domino's Pizza,Consumer Discretionary,383.46,25,"9,586.50"
6,F,Ford Motor Company,Consumer Discretionary,8.79,1,8.79
7,NFLX,Netflix Inc.,Communication Services,540.73,1,540.73
8,POOL,Pool Corporation,Consumer Discretionary,372.50,11,"4,097.50"
9,TTWO,Take-Two Interactive,Communication Services,207.79,2,415.58


In [99]:
print (f'Total allocated amount = ${total_allocation:,.2f}')
print (f'Amount remaining: ${leftover:,.2f}')
ef.portfolio_performance(verbose=True)

Total allocated amount = $49,999.90
Amount remaining: $0.10
Expected annual return: 136.7%
Annual volatility: 25.5%
Sharpe Ratio: 5.28


(1.3672872952337365, 0.25519869984984, 5.279365827594287)